In [35]:
import pandas as pd
import bs4
import json
import re
import tqdm

import scipy.stats as st
import numpy as np

import bokeh
from bokeh.plotting import show as show_interactive
from bokeh.plotting import output_file, output_notebook
from bokeh.layouts import column, row
from bokeh.models import CustomJS, TextInput, LassoSelectTool, Select, MultiSelect, ColorBar, Legend, LegendItem
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, Button, HTMLTemplateFormatter
from bokeh.events import SelectionGeometry
from bokeh.transform import linear_cmap, jitter
from bokeh.models import FactorRange


In [36]:
complete_annotation = pd.read_csv('../eggnog/complete_eggnog_annotation.csv')
go_df = pd.read_csv('./go_annotations.csv')
kegg_df = pd.read_csv('./kegg_annotations.csv')
ec_df = pd.read_csv('./ec_annotations.csv')

In [37]:
go_df.head()

,GOs,GOs_description,GOs_definition,GOs_obsolete
0,-,-,-,-
1,GO:0000001,mitochondrion inheritance,"The distribution of mitochondria, including th...",False
2,GO:0000002,mitochondrial genome maintenance,The maintenance of the structure and integrity...,False
3,GO:0000003,reproduction,The production of new individuals that contain...,False
4,GO:0000011,vacuole inheritance,The distribution of vacuoles into daughter cel...,False


In [38]:
go_df['GOs_description'].loc[go_df['GOs'] == 'GO:0000001'].values[0]

'mitochondrion inheritance'

In [39]:
kegg_df.head()

,KEGG_ko,KEGG_ko_description
0,-,-
1,ko:K00002,alcohol dehydrogenase (NADP+)
2,ko:K00006,glycerol-3-phosphate dehydrogenase (NAD+)
3,ko:K00011,aldehyde reductase
4,ko:K00012,UDPglucose 6-dehydrogenase


In [40]:
ec_df.head()

,EC,EC_description
0,-,-
1,1.1.1.1,alcohol dehydrogenase
2,1.1.1.105,all-trans-retinol dehydrogenase (NAD(+))
3,1.1.1.141,15-hydroxyprostaglandin dehydrogenase (NAD(+))
4,1.1.1.178,3-hydroxy-2-methylbutyryl-CoA dehydrogenase


In [41]:
# As of 2020 https://www.ncbi.nlm.nih.gov/research/cog/
COG_dict = {
    "A" : "RNA processing and modification",
    "B" : "Chromatin structure and dynamics",
    "C" : "Energy production and conversion",
    "D" : "Cell cycle control, cell division, chromosome partitioning",
    "E" : "Amino acid transport and metabolism",
    "F" : "Nucleotide transport and metabolism",
    "G" : "Carbohydrate transport and metabolism",
    "H" : "Coenzyme transport and metabolism",
    "I" : "Lipid transport and metabolism",
    "J" : "Translation, ribosomal structure and biogenesis",
    "K" : "Transcription",
    "L" : "Replication, recombination, and repair",
    "M" : "Cell wall/membrane/envelope biogenesis",
    "N" : "Cell motility",
    "O" : "Posttranslational modification, protein turnover, chaperones",
    "P" : "Inorganic ion transport and metabolism",
    "Q" : "Secondary metabolites biosynthesis, transport and catabolism",
    "T" : "Signal transduction mechanisms",
    "U" : "Intracellular trafficking, secretion, and vesicular transport",
    "V" : "Defense mechanisms",
    "W" : "Extracellular structures",
    "X" : "Mobilome: prophages, transposons",
    "Y" : "Nuclear structure",
    "Z" : "Cytoskeleton",
    "R" : "General function prediction only",
    "S" : "Function unknown",
}

In [42]:
def term_count_dict_from_annotation_df(annot_df, term_column):
    
    column = annot_df[term_column].values
    
    funct_terms = []
    for entry in column:
        if entry != '-':
            if term_column == 'COG_category':
                # terms = [f'{e}: {COG_dict[e]}' for e in entry]
                terms = list(entry)
            else:
                terms = entry.split(',')
            for t in terms:
                funct_terms.append(t)

#     len(terms)
    
    term_count_dict = {}
    
    for t in funct_terms:
        count = term_count_dict.get(t, 0)
        count += 1
        term_count_dict[t] = count
        
    return term_count_dict

In [43]:
def enrichment_analysis(module, leiden_label_df, phases, background_annotation, term_column):
    
    module_ttids = leiden_label_df.loc[leiden_label_df[f'leiden_label_{phases}'] == module]['TTHERM_ID'].values
    
    module_annotation = background_annotation.loc[background_annotation['TTHERM_ID'].isin(module_ttids)]
    
    background_term_dict = term_count_dict_from_annotation_df(background_annotation, term_column)
    module_term_dict = term_count_dict_from_annotation_df(module_annotation, term_column)
    
    bs = []
    ps = []
    folds = []
    terms = []
    
    for t, module_count in module_term_dict.items():
        
        background_count = background_term_dict[t]
        module_size = len(module_annotation)
        background_size = len(background_annotation)
        
        standard_contingency_table = [
                                [module_count, background_count - module_count], 
                                [module_size - module_count, background_size - module_size - (background_count - module_count)]
                            ]
        
        # The -1 and +1 make this more conservative (see explanation from the DAVID database: 
        # https://david.ncifcrf.gov/helps/functional_annotation.html#geneenrich)
        conservative_contingency_table = [
                                [module_count - 1, background_count - module_count + 1], 
                                [module_size - module_count, background_size - module_size - (background_count - module_count)]
                            ]
        
        
        odds, p_standard = st.fisher_exact(standard_contingency_table, 'greater')
        odds, p_conservative = st.fisher_exact(conservative_contingency_table, 'greater')
        
        p_reasonable = np.mean([p_standard, p_conservative])
        
        bonferroni  = p_reasonable * len(module_term_dict)

        fold_enrichment = (module_count/module_size) / (background_count/background_size)

        if bonferroni <= 0.05:
            
            ps.append(p_reasonable)
            bs.append(bonferroni)
            folds.append(fold_enrichment)
            terms.append(t)
            
#         else:
#             ps.append('')
#             bs.append('')
#             folds.append('')
#             terms.append('')
            
    return ps, bs, folds, terms
            
            

In [44]:
def get_GO_info(go_term):
    
    name = go_df['GOs_description'].loc[go_df['GOs'] == go_term].values[0]
    
    definition = go_df['GOs_definition'].loc[go_df['GOs'] == go_term].values[0]
    
    obsolete = go_df['GOs_obsolete'].loc[go_df['GOs'] == go_term].values[0]
    
    return name, definition, obsolete

In [45]:
get_GO_info('GO:0000001')

('mitochondrion inheritance',
 'The distribution of mitochondria, including the mitochondrial genome, into daughter cells after mitosis or meiosis, mediated by interactions between mitochondria and the cytoskeleton.',
 'False')

In [46]:
def get_KEGG_info(term):
    return kegg_df['KEGG_ko_description'].loc[kegg_df['KEGG_ko'] == term].values[0]

In [47]:
get_KEGG_info('ko:K00002')

'alcohol dehydrogenase (NADP+)'

In [48]:
def get_EC_info(term):
    return ec_df['EC_description'].loc[ec_df['EC'] == term].values[0]

In [49]:
get_EC_info('1.1.1.141')

'15-hydroxyprostaglandin dehydrogenase (NAD(+))'

In [50]:
def write_enrichment(lldf, phases, out_prefix, term_column):
    
    with open(f'../enrichment/{out_prefix}_enrichment.txt', 'w') as fl:
    
        for m in tqdm.tqdm(sorted(lldf[f'leiden_label_{phases}'].unique())):

            # print(f'Module {m}')
            ps, bs, folds, terms = enrichment_analysis(m, lldf, phases, complete_annot, term_column)

            fl.write(f'Module {m}\n')
            for b, fold, t in zip(bs, folds, terms):
                
                if term_column == 'COG_category':
                    
                    definition = COG_dict[t]
                    t = f'{t}: {definition}'
                
                elif term_column == 'GOs':
                    name, definition, obsolete = get_GO_info(t)
                    if obsolete:
                        t = f'{name.capitalize()} ({t}): {definition} (obsolete)'
                    else:
                        t = f'{name.capitalize()} ({t}): {definition}'
                        
                elif term_column.split('_')[0] == 'KEGG':
                    name = get_KEGG_info(t)
                    
                    t = f'{t}: {name}'
                    
                elif term_column == 'EC':
                    
                    name = get_EC_info(t)
                    t = f'{t}: {name}'
                    
                        
                # print(f"{t}\nFold-enrichment: {fold:.02f}\nBonferroni-corrected p-value: {b:.02e}\n\n")

                fl.write(f'{t}\nFold-enrichment: {fold:.02f}\nBonferroni-corrected p-value: {b:.02e}\n\n')

In [51]:
def get_enrichment_df(lldf, phases, background_annotation, term_columns=['COG_category', 'GOs', 'KEGG_ko', 'EC'], outfile=None):
    
    module_dfs = []
    
    for m in tqdm.tqdm(sorted(lldf[f'leiden_label_{phases}'].unique())):
        
        term_dfs = []

        for tc in term_columns:
        
            ps, bs, folds, terms = enrichment_analysis(m, lldf, phases, background_annotation, tc)

            # fl.write(f'Module {m}\n')
            
            info = []

            if tc == 'GOs':

                for t in terms:
                    name, definition, obsolete = get_GO_info(t)
                    if obsolete:
                        info.append(f'{name.capitalize()}: {definition} (obsolete)')
                    else:
                        info.append(f'{name.capitalize()}: {definition}')
                        
            elif tc == 'COG_category':
                for t in terms:
                    info.append(COG_dict[t])
                                
            elif tc == 'KEGG_ko':
                for t in terms:
                    info.append(get_KEGG_info(t))
                    
            elif tc == 'EC':
                for t in terms:
                    info.append(get_EC_info(t))
                    
            term_df = pd.DataFrame({'module': [m]*len(terms),
                                    'term': terms,
                                    'info': info,
                                    'fold_change': folds,
                                    'bonferroni': bs})
            
            term_dfs.append(term_df)
            
        module_df = pd.concat(term_dfs)
        
        module_dfs.append(module_df)
        
    all_enrichment_df = pd.concat(module_dfs)
    
    if outfile:
        all_enrichment_df.to_csv(outfile, index=False)
    
    return all_enrichment_df
        
        
                        
            
                    
            
        
#         for b, fold, t in zip(bs, folds, terms):

#             if term_column == 'GOs':
#                 name, definition, obsolete = get_GO_info(t)
#                 if obsolete:
#                     t = f'{name.capitalize()} ({t}): {definition} (obsolete)'
#                 else:
#                     t = f'{name.capitalize()} ({t}): {definition}'

#             if term_column.split('_')[0] == 'KEGG':
#                 dfs = pd.read_html(f'https://www.genome.jp/entry/{t}')

#                 data = dfs[4]
#                 entry = data.loc[data[0] == 'Entry'][1].values[0]
#                 name = data.loc[data[0] == 'Name'][1].values[0]

#                 t = f'{t}: {name}'

In [52]:
complete_annotation = pd.read_csv('../eggnog/complete_eggnog_annotation.csv')

In [53]:
lldf_nn3 = pd.read_csv('../embedding/test_nn3_leiden_label_df_round_1.csv')

In [54]:
gene_list_test = ['TTHERM_000013409', 'TTHERM_01321550', 'TTHERM_00011710', 'TTHERM_00321680', 'TTHERM_00355700', 'TTHERM_00938950', 'TTHERM_01372820', 'TTHERM_00013410', 'TTHERM_00390080', 'TTHERM_00516380', 'TTHERM_00038880', 'TTHERM_00059370', 'TTHERM_00473020', 'TTHERM_00497590', 'TTHERM_00558350', 'TTHERM_00052190', 'TTHERM_00392790', 'TTHERM_00410180', 'TTHERM_00685980', 'TTHERM_00445920', 'TTHERM_00471040', 'TTHERM_00140780', 'TTHERM_00145480', 'TTHERM_00321720', 'TTHERM_00628650', 'TTHERM_00526730', 'TTHERM_01156770', 'TTHERM_00312200', 'TTHERM_01332070', 'TTHERM_00318900', 'TTHERM_00340180', 'TTHERM_00592740', 'TTHERM_00440600', 'TTHERM_01321570', 'TTHERM_00537380', 'TTHERM_00585170', 'TTHERM_01197130', 'TTHERM_00554390', 'TTHERM_00649180', 'TTHERM_00691410', ]
for gene in gene_list_test:
    print((lldf_nn3.loc[lldf_nn3['TTHERM_ID'] == gene])['leiden_label_rna_seq'].values[0])

1295
684
412
684
508
412
324
2
706
27
971
369
412
412
422
412
775
124
124
124
124
693
412
772
419
1142
542
422
784
188
135
1343
324
870
124
387
59
412
124
87


In [55]:
lldf_nn3['leiden_label_rna_seq'].unique()

array([1365, 1364, 1363, ...,    2,    1,    0])

In [56]:
# %pdb

In [57]:
enrich_full_nn3 = get_enrichment_df(lldf_nn3, 'rna_seq', complete_annotation, outfile='./test_nn3_full_enrichment.csv')

100%|██████████| 1366/1366 [02:10<00:00, 10.46it/s]


In [58]:
enrich_full_nn3 = pd.read_csv('./test_nn3_full_enrichment.csv')

In [59]:
enrich_full_nn3.head(40)

,module,term,info,fold_change,bonferroni
0,0.0,L,"Replication, recombination, and repair",13.352657,1.427466e-05
1,0.0,K,Transcription,7.452038,3.511865e-02
2,0.0,B,Chromatin structure and dynamics,20.206233,7.586105e-03
3,4.0,L,"Replication, recombination, and repair",16.600601,2.101164e-06
4,9.0,Z,Cytoskeleton,8.132559,4.675158e-02
5,10.0,O,"Posttranslational modification, protein turnov...",3.343265,4.137196e-02
6,12.0,L,"Replication, recombination, and repair",24.839869,5.451007e-11
7,12.0,3.6.4.12,DNA helicase,38.614706,7.964236e-03
8,20.0,Z,Cytoskeleton,12.938162,4.227486e-04
9,20.0,D,"Cell cycle control, cell division, chromosome ...",11.983388,3.690332e-03


In [60]:
for term in enrich_full_nn3['term'].unique():
    if 'ko:' not in term and 'GO:' not in term and '.' not in term:
        print(term)

L
K
B
Z
O
D
J
C
A
T
Y
F
S
P
G
I
N
U
E


In [61]:
print(max(enrich_full_nn3['module'].values))
print(min(enrich_full_nn3['module'].values))

not_enriched_mod_list = []
enriched_mod_list = []

for num in range(int(max(enrich_full_nn3['module'].values)) + 1):
    if num not in enrich_full_nn3['module'].values:
        not_enriched_mod_list.append(num)
    if num in enrich_full_nn3['module'].values:
        enriched_mod_list.append(num)

print(len(not_enriched_mod_list) + len(enriched_mod_list))
print(max(not_enriched_mod_list + enriched_mod_list))
print()

print(set(range(int(max(enrich_full_nn3['module'].values)) + 1)) - set(enrich_full_nn3['module'].values))
print(len(set(enrich_full_nn3['module'].values)))
print(len(enriched_mod_list))
print(len(not_enriched_mod_list))

1337.0
0.0
1338
1337

{1, 2, 3, 5, 6, 7, 8, 11, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 51, 53, 55, 56, 57, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 72, 73, 75, 76, 77, 78, 79, 81, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 99, 101, 103, 104, 105, 106, 107, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 147, 150, 151, 152, 153, 154, 155, 156, 157, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 170, 171, 172, 173, 176, 177, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 192, 193, 194, 195, 196, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 229, 230, 231, 232, 233, 234, 235, 236, 238, 239, 240, 242, 243, 244, 245, 246, 247, 248, 250, 251, 252, 253, 254, 2

In [62]:
print(not_enriched_mod_list)

[1, 2, 3, 5, 6, 7, 8, 11, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 51, 53, 55, 56, 57, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 72, 73, 75, 76, 77, 78, 79, 81, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 99, 101, 103, 104, 105, 106, 107, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 147, 150, 151, 152, 153, 154, 155, 156, 157, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 170, 171, 172, 173, 176, 177, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 192, 193, 194, 195, 196, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 229, 230, 231, 232, 233, 234, 235, 236, 238, 239, 240, 242, 243, 244, 245, 246, 247, 248, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259

In [63]:
enrich_full_nn3.loc[enrich_full_nn3['module']==72]

,module,term,info,fold_change,bonferroni


In [64]:
enrich_full_nn3['module'].unique()

array([   0.,    4.,    9.,   10.,   12.,   20.,   25.,   26.,   30.,
         38.,   49.,   50.,   52.,   54.,   58.,   70.,   71.,   74.,
         80.,   82.,   96.,   97.,   98.,  100.,  102.,  108.,  132.,
        146.,  148.,  149.,  158.,  169.,  174.,  175.,  182.,  190.,
        191.,  197.,  209.,  228.,  237.,  241.,  249.,  262.,  266.,
        276.,  284.,  290.,  307.,  310.,  319.,  327.,  334.,  345.,
        349.,  360.,  362.,  365.,  380.,  386.,  403.,  412.,  418.,
        447.,  452.,  484.,  489.,  498.,  517.,  534.,  545.,  551.,
        564.,  577.,  580.,  598.,  622.,  635.,  638.,  641.,  661.,
        677.,  694.,  697.,  712.,  725.,  739.,  746.,  756.,  786.,
        788.,  795.,  810.,  823.,  835.,  844.,  850.,  853.,  872.,
        919.,  923.,  927.,  934.,  937.,  938.,  949.,  958.,  991.,
        995.,  996., 1006., 1007., 1012., 1015., 1060., 1079., 1101.,
       1167., 1191., 1194., 1195., 1219., 1243., 1249., 1267., 1269.,
       1271., 1279.,

In [65]:
lldf_nn3['leiden_label_rna_seq'].unique()

array([1365, 1364, 1363, ...,    2,    1,    0])

In [66]:
lldf_nn3.loc[lldf_nn3['TTHERM_ID'] == 'TTHERM_01386050']

,TTHERM_ID,leiden_label_rna_seq
9592,TTHERM_01386050,622


In [67]:
gene_list_test = ['TTHERM_000013409', 'TTHERM_01321550', 'TTHERM_00011710', 'TTHERM_00321680', 'TTHERM_00355700', 'TTHERM_00938950', 'TTHERM_01372820', 'TTHERM_00013410', 'TTHERM_00390080', 'TTHERM_00516380', 'TTHERM_00038880', 'TTHERM_00059370', 'TTHERM_00473020', 'TTHERM_00497590', 'TTHERM_00558350', 'TTHERM_00052190', 'TTHERM_00392790', 'TTHERM_00410180', 'TTHERM_00685980', 'TTHERM_00445920', 'TTHERM_00471040', 'TTHERM_00140780', 'TTHERM_00145480', 'TTHERM_00321720', 'TTHERM_00628650', 'TTHERM_00526730', 'TTHERM_01156770', 'TTHERM_00312200', 'TTHERM_01332070', 'TTHERM_00318900', 'TTHERM_00340180', 'TTHERM_00592740', 'TTHERM_00440600', 'TTHERM_01321570', 'TTHERM_00537380', 'TTHERM_00585170', 'TTHERM_01197130', 'TTHERM_00554390', 'TTHERM_00649180', 'TTHERM_00691410', ]
for gene in gene_list_test:
    print((lldf_nn3.loc[lldf_nn3['TTHERM_ID'] == gene])['leiden_label_rna_seq'].values[0])

1295
684
412
684
508
412
324
2
706
27
971
369
412
412
422
412
775
124
124
124
124
693
412
772
419
1142
542
422
784
188
135
1343
324
870
124
387
59
412
124
87


In [68]:
len(set(enrich_full_nn3['module'].values))

130

end here. Return to embedding notebook!